In [255]:
import os
import sys
import pandas as pd
import numpy as np

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

tf.__version__

'2.1.0-rc1'

In [256]:
# add project to the Pyithon path
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [257]:
# from components.model import get_model

In [258]:
column_names = ["col_0","complaint_id","product","sub_product","issue","sub_issue","state","zip_code","date_received","date_sent_to_company","company","company_response","timely_response","consumer_disputed"]
feature_names = ["product", "sub_product", "issue", "sub_issue", "state", "zip_code", "company", "company_response", "timely_response", "consumer_disputed"]
cat_features = ["product", "sub_product", "state", "zip_code", "company_response", "timely_response", "consumer_disputed"]
text_features = ["issue", "sub_issue", "company",]
LABEL_COLUMN = 'consumer_disputed'

batch_size = 32
EVAL_SAMPLES = 2000

In [289]:
# load the dataset
def load_dataset(filename):
    df = pd.read_csv(filename, usecols=feature_names, na_values=0, keep_default_na=False)
    df.consumer_disputed.replace("", "No", inplace=True)

    for feature in cat_features:
        df[feature] = df[feature].astype("category").cat.codes

    for feature in text_features:
        df[feature] = df[feature].apply(lambda x: x.strip()).replace('', 'UNK')

    X = [pd.np.asarray(df[feature_name].values).reshape(-1) for feature_name in feature_names][:-1]
    y = np.asarray(df.pop("consumer_disputed"), dtype=np.uint8).reshape(-1)
    return X, y
 
X, y = load_dataset("../data/26k-consumer-complaints-modified.csv")


In [290]:

# def get_dataset(file_path, **kwargs):
#     dataset = tf.data.experimental.make_csv_dataset(
#         file_path,
#         column_names=column_names,
#         select_columns=feature_names,
#         batch_size=batch_size,
#         label_name=LABEL_COLUMN,
#         na_value="?",
#         num_epochs=1,
#         ignore_errors=True, 
#         **kwargs)
#     return dataset
# dataset = get_dataset("../data/26k-consumer-complaints-modified.csv")
# features, labels = next(iter(dataset))
# print(features)
# print(labels)

In [299]:
# import tensorflow as tf
# import tensorflow_hub as hub

# def get_model(show_summary=True):
#     """
#     Function defines a Keras model and returns the model as Keras object
#     """
#     # how do you get the size of the embedding?? Why use an embedding when there are only 2 dimensions to start with?
#     # in fact, why 2 dimensions for 0/1?

#     # "product", "sub_product", "issue", "sub_issue", "state", "zip_code", "company", "company_response", "timely_response"
#     #    x0            x1           x2        x3         x4        x5          x6             x7                 x8

#     # Cat: "product", "sub_product", "state", "zip_code", "company_response", "timely_response", "consumer_disputed"

#     # categorical inputs
#     input_product = tf.keras.Input(shape=(1,), name="product_xf")
#     input_sub_product = tf.keras.Input(shape=(1,), name="sub_product_xf")
#     input_state = tf.keras.Input(shape=(1,), name="state_xf")
#     input_zip_code = tf.keras.Input(shape=(1,), name="zip_code_xf")
#     input_company_response = tf.keras.Input(shape=(1,), name="company_response_xf")
#     input_timely_response = tf.keras.Input(shape=(1,), name="timely_response_xf")

#     # text inputs
#     module_url = "https://tfhub.dev/google/nnlm-en-dim128/2"
#     embed = hub.KerasLayer(module_url)
#     input_issue = tf.keras.Input(shape=(1,), name="issue_xf", dtype=tf.string)
#     input_sub_issue = tf.keras.Input(shape=(1,), name="sub_issue_xf", dtype=tf.string)
#     input_company_name = tf.keras.Input(shape=(1,), name="company_xf", dtype=tf.string)

#     def cnn_layers(x):
#         reshaped_x = tf.reshape(x, [-1])
#         embedded_x = embed(reshaped_x) 
#         conv_x = tf.keras.layers.Reshape((1, 128), input_shape=(128,))(embedded_x)
#         conv_x = tf.keras.layers.Conv1D(32, 1, activation='relu')(conv_x)
#         conv_x = tf.keras.layers.GlobalMaxPooling1D()(conv_x)
#         conv_x = tf.keras.layers.Dense(10, activation='relu')(conv_x)
#         return conv_x

#     # convert to embeddings
#     x0 = tf.keras.layers.Embedding(12, 3)(input_product)
#     x0 = tf.keras.layers.Reshape((3, ), input_shape=(1, 3))(x0)

#     x1 = tf.keras.layers.Embedding(60, 5)(input_sub_product)
#     x1 = tf.keras.layers.Reshape((5, ), input_shape=(1, 5))(x1)

#     x4 = tf.keras.layers.Embedding(60, 5)(input_state)
#     x4 = tf.keras.layers.Reshape((5, ), input_shape=(1, 5))(x4)

#     x5 = tf.keras.layers.Embedding(10000, 5)(input_zip_code)
#     x5 = tf.keras.layers.Reshape((5, ), input_shape=(1, 5))(x5)

#     x7 = tf.keras.layers.Embedding(10, 3)(input_company_response)
#     x7 = tf.keras.layers.Reshape((3, ), input_shape=(1, 3))(x7)

#     x8 = tf.keras.layers.Embedding(2, 1)(input_timely_response)
#     x8 = tf.keras.layers.Reshape((1, ), input_shape=(1, 2))(x8)

#     x_feed_forward = tf.keras.layers.concatenate(
#         [x0, x1, x4, x5, x7, x8])

#     x = tf.keras.layers.Dense(100, activation='relu')(x_feed_forward)
#     x = tf.keras.layers.Dense(50, activation='relu')(x)
#     x = tf.keras.layers.Dense(10, activation='relu')(x)

#     conv_issue = cnn_layers(input_issue)
#     conv_sub_issue = cnn_layers(input_sub_issue)
#     conv_company = cnn_layers(input_company_name)

#     x = tf.keras.layers.concatenate([x, conv_company, conv_issue, conv_sub_issue])
#     x = tf.keras.layers.Dense(10, activation='relu')(x)
#     output = tf.keras.layers.Dense(1, activation='sigmoid')(x) 

#     _inputs = [input_product, input_sub_product,
#               input_issue, input_sub_issue, 
#               input_state, input_zip_code, input_company_name, 
#               input_company_response, input_timely_response] 

#     keras_model = tf.keras.models.Model(_inputs, output)
#     keras_model.compile(optimizer='rmsprop',
#                      loss='binary_crossentropy',  # categorical_crossentropy
#                      metrics=[
#                          'accuracy', tf.keras.metrics.TruePositives(), 
#                          tf.keras.metrics.TrueNegatives(), tf.keras.metrics.FalsePositives(), 
#                          tf.keras.metrics.FalseNegatives()]
#                         )
#     if show_summary:
#         keras_model.summary()

#     return keras_model

In [300]:
model = get_model()

Model: "model_31"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
product_xf (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
sub_product_xf (InputLayer)     [(None, 1)]          0                                            
__________________________________________________________________________________________________
state_xf (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
zip_code_xf (InputLayer)        [(None, 1)]          0                                            
___________________________________________________________________________________________

In [303]:
model.fit(x=X, y=y, batch_size=batch_size, validation_split=0.25, epochs=4)

Train on 21117 samples, validate on 7039 samples
Epoch 1/4
21117/21117 [==============================] - 3s 120us/sample - loss: 0.3926 - accuracy: 0.8442 - true_positives_13: 0.0000e+00 - true_negatives_13: 17827.0000 - false_positives_13: 0.0000e+00 - false_negatives_13: 3290.0000 - val_loss: 0.4890 - val_accuracy: 0.7986 - val_true_positives_13: 0.0000e+00 - val_true_negatives_13: 5621.0000 - val_false_positives_13: 0.0000e+00 - val_false_negatives_13: 1418.0000
Epoch 2/4
21117/21117 [==============================] - 2s 98us/sample - loss: 0.3761 - accuracy: 0.8452 - true_positives_13: 36.0000 - true_negatives_13: 17812.0000 - false_positives_13: 15.0000 - false_negatives_13: 3254.0000 - val_loss: 0.4858 - val_accuracy: 0.8000 - val_true_positives_13: 30.0000 - val_true_negatives_13: 5601.0000 - val_false_positives_13: 20.0000 - val_false_negatives_13: 1388.0000
Epoch 3/4
21117/21117 [==============================] - 2s 97us/sample - loss: 0.3576 - accuracy: 0.8535 - true_positiv